# Information Retrieval
Author: Yiming Zhang 
Stuent ID: 889262

It implements the document retrieval and sentence retrieval.
I builds a dictionary to store the page identifier and sentence index.
Then use the rule based information extraction function to retrievel the document.
And many methods such as tfidf, BOW are used to retrievel the sentence.

## Library

In [1]:
import os
import gc
import json
import pickle
import datetime
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import groupby
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import coo_matrix
import multiprocessing
from multiprocessing import Manager,Pool
import copy

## Load  dataframe
The dataframe is the result from "Process_New_wiki.ipynb".

In [2]:
# load_processed_corpus_df = pd.read_pickle("./processed_corpus.pkl")
load_processed_corpus_df = pd.read_csv("./new_wiki.csv")
print(load_processed_corpus_df.shape)
load_processed_corpus_df.head(10)

(25248397, 3)


,page_identifier,sentence_number,text
0,42nd_Street_Ferry,0,42nd_Street_Ferry 0 42nd Street Ferry 42nd Str...
1,42nd_Street_Ferry,3,42nd_Street_Ferry 3 42nd Street Ferry 42nd Str...
2,42nd_Street_Ferry,5,42nd_Street_Ferry 5 42nd Street Ferry 42nd Str...
3,Aeroflot-Plus,0,Aeroflot-Plus 0 Aeroflot-Plus Aeroflot-Plus wa...
4,A._U._Wyman,0,A._U._Wyman 0 A. U. Wyman Albert U. Wyman 1833...
5,A._U._Wyman,3,A._U._Wyman 3 A. U. Wyman Wyman was raised in ...
6,A._U._Wyman,4,A._U._Wyman 4 A. U. Wyman After completing a c...
7,A._U._Wyman,5,A._U._Wyman 5 A. U. Wyman He later went into b...
8,A._U._Wyman,8,"A._U._Wyman 8 A. U. Wyman In 1863 , Wyman beca..."
9,A._U._Wyman,9,A._U._Wyman 9 A. U. Wyman He was then appointe...


## Load  traning, dev, test dataset.

In [3]:
with open('devset.json', 'r') as f1:  # load dev dataset
        dev_data = json.load(f1) 
print("Length of the dev data is: " + str(len(dev_data)))

with open('devset_result.json', 'r') as f2:  # store result 
        res_data = json.load(f2) 
print("Length of the dev result data is: " + str(len(res_data)))

with open('test-unlabelled.json', 'r') as f3:  # store result 
     test_data = json.load(f3) 
print("Length of the test data is: " + str(len(test_data)))

Length of the dev data is: 5001
Length of the dev result data is: 5001
Length of the test data is: 14997


## Preprocess
Remove stop words, lemma, tokenzition.

In [4]:
nltk.download('stopwords')

lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def pre_process(comment) -> str:
    # lower cased
    comment = comment.lower()
    # tokenize
    words =  nltk.tokenize.word_tokenize(comment)
    # lemmatize 
    words = [lemmatize(w) for w in words]
    
    stop_words = nltk.corpus.stopwords.words('english')
    words = [w for w in words if not w in stop_words]
    processed_comment = " ".join(words)
    return processed_comment


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhangyiming/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Build a dictionary to store index in dataframe ( only run once)
## Deal with LRB in dictionary. (only run once)
The code of building the dictionary is in the comment below.
For convience, I just save the dicitoanry and load it.

In [5]:
with open('lrb_dictionary.json', 'r') as f4:  # load lrb dictioanry json
        lrb_dictionary = json.load(f4) 

# method one 
# page_dictionary = {}

# for index, row in load_processed_corpus_df.iterrows(): 
#     if isinstance(row['page_identifier'], float):
#         continue
#     page_dictionary.setdefault(row['page_identifier'],[]).append(index)

# def solve_LRB(dictionary):
#     for key, value in dictionary.items():
#         if '_-LRB' in key:
#             if key.split('_-LRB')[0] in dictionary:
#                 for v in value:
#                     dictionary[key.split('_-LRB')[0]].append(v)

#     return dictionary

# lrb_dictionary = solve_LRB(page_dictionary)

# method two
# page_dictionary = {}

# for index, row in load_processed_corpus_df.iterrows(): 
#     if isinstance(row['page_identifier'], float):
#         continue
#     page_dictionary.setdefault(row['page_identifier'],[]).append(index)

# def solve_LRB(dictionary):
#     lrb_dictionary = copy.deepcopy(dictionary)
#     for key, value in dictionary.items():
#         if '_-LRB' in key:
#             if key.split('_-LRB')[0] in lrb_dictionary:
#                 lrb_dictionary[key.split('_-LRB')[0]].extend(value)
#             else:
#                 lrb_dictionary.update({
#                     key.split('_-LRB')[0]: value
#                 })
#     return lrb_dictionary

# lrb_dictionary = solve_LRB(page_dictionary)


In [ ]:
# with open('lrb_dictionary_new.json', 'w') as f:
#     json.dump(lrb_dictionary, f, indent = 4)

## Delete keys in dictionary.
Some words in the page identifeir are irrelvant to the evidnece, so we delete some after observation.

In [6]:
delete_words_list = ['The','Part', 'Most','Water', 'How', 'Love','Speech','American','President','German','Irish',
                     'Indian','Spanish','Japan','Califorina','Americans','Chinese','British','Monday','Tuesday',
                    'Wednesday','Thursday', 'Friday','Saturday', 'Sunday','January','February','March','April',
                     'May', 'June', 'July', 'August', 'September', 'October', 'November','December','Russian']

for word in delete_words_list:
    try:
        del lrb_dictionary[word]
    except KeyError:
        pass
    

In [7]:
# page keys are the page identifier, keys of the page_dictionary
page_keys = list(lrb_dictionary.keys())

## Retrieve the sentence text.

In [8]:
# input: a page identifier string, keys are all the keys of the page_dictionary
# output: return a list of strings which contains all the relevanted sentence text,also with page_identifier and sentence number so
# that it can be written as evidence part.

def retrieve_sentenceText(claim_word,page_keys,page_dictionary,df):
    retrieved_sentence = []
    if claim_word in page_keys:
        retrieved_index = page_dictionary[claim_word]      # all indexes in the dataframe
        for index in retrieved_index:
             # retrieve all the raw doc txt
            retrieved_sentence.append(df.loc[index, 'text'])
    return retrieved_sentence

## Rule on Claim: find the Upper cased words.

Rule1: First Continuous Upper words.

In [9]:
def find_upper_word(claim, page_keys):
    res_list = []
    res_index = []      
    words = claim.split()
    
    start = ""
    temp = ""
    i = 0
    while i < len(words):         
        # first step: find uppercase word in the claim
        if words[i][0].isupper():
            temp = words[i]        
            start = temp  # start(as a cache)                        
            for j in range(i,len(words)-1):
                temp = temp + '_' + words[j+1]
#                 print(temp)
                if temp in page_keys:
                    start = temp  # matchs the word as long as possible
                    i = j + 1
                
                if j - i > 2:
                    break
                
            res_list.append(start)  
        i += 1                    
    return res_list


In [21]:
# test_claim1 = "International students come to the University of Mississippi from 90 cities"
# print(find_upper_word(test_claim1,page_keys))
# test_claim2 = "Home for the Holidays stars an American actress"
# print(find_upper_word(test_claim2,page_keys))
# test_claim3 = "Water is part of the History of Earth"
# print(find_upper_word(test_claim3,page_keys))
# test_claim4 = "How to Train Your Dragon 2 used real dragons"
# print(find_upper_word(test_claim4,page_keys))
# test_claim5 = "The Armenian Genocide was the extermination of Armenians who were mostly Turkish citizens."
# print(find_upper_word(test_claim5,page_keys))
# test_claim6 = "Boyhood is about Mason Evans, Jr's childhood."
# print(find_upper_word(test_claim6,page_keys))
# test_claim7 = "L.A. Reid has served as the president of a record label."
# print(find_upper_word(test_claim7,page_keys))
# test_claim8 = "there are no musical or creative works in existence that have been created by Philip Glass"
# print(find_upper_word(test_claim8,page_keys))

test_claim9 = "Home for the Holidays stars an American actress."
print(find_upper_word(test_claim9,page_keys))
test_claim10 = "Home for the Holidays 1995 film The film stars Holly Hunter"
print(find_upper_word(test_claim10,page_keys))

['Home_for_the_Holidays', 'American']
['Home_for_the_Holidays', 'The', 'Holly_Hunter']


In [ ]:
# TESTING LRB
# print(find_upper_word("Savages was exclusively a German film."))
# print(retrieve_sentenceText(find_upper_word("Savages was exclusively a German film.")[0], page_keys,lrb_dictionary,load_processed_corpus_df))

## Method1 : TFIDF and SVD

In [ ]:
def retrieval_evidence_func(query,page_keys,page_dictionary,load_processed_corpus_df):
        
    res = []
    # Determine if the first word in query is "There" and "A"
    if query.split()[0] == "There":
        query = query.replace("There","there") 
        
    if query.split()[0] == "A":
        query = query.replace("A","a") 
    
    # remove all 's 
    for word in query.split():
        if (word[-2:len(word)]) == "'s":
            query = query.replace(word,word[:-2])
            
    query_corpus = []
    if len(find_upper_word(query,page_keys)) >= 1:
        for query_word in find_upper_word(query,page_keys):
            if query_word[-1:len(query_word)] == ".":
                query_word = query_word[:-1]
            if query_word[-1:len(query_word)] == ",":
                query_word = query_word[:-1]
            query_corpus.extend(retrieve_sentenceText(query_word, page_keys,page_dictionary,load_processed_corpus_df))
        
        if len(query_corpus) == 1:
            res.append([query_corpus[0].split()[0],int(query_corpus[0].split()[1])])
        else:
            # build a tfidf model on the query corpus
            tfidf_vectorizer = TfidfVectorizer()
            tfidf_matrix = tfidf_vectorizer.fit_transform(query_corpus)
            tfidf_matrix = tfidf_matrix.T

            # apply svd
            K= 2 # number of desirable features 
            U, s, VT = np.linalg.svd(tfidf_matrix.toarray())
            #tfidf_reduced = np.dot(U[:,:K], np.dot(np.diag(s[:K]), VT[:K, :]))
            terms_rep = np.dot(U[:,:K], np.diag(s[:K]))
            docs_rep = np.dot(np.diag(s[:K]), VT[:K, :]).T

            # calculate query rep
            query_rep = []
            for q in pre_process(query).split():
                if q in tfidf_vectorizer.vocabulary_:
                    query_rep.append(tfidf_vectorizer.vocabulary_[q])
                else:
                    continue
            query_rep = np.mean(terms_rep[query_rep],axis=0)

            # calculate cosine similarity between the query and retrieved sentences
            query_doc_cos_dist = []
            for doc_rep in docs_rep:
                query_doc_cos_dist.append(cosine(query_rep, doc_rep))

            query_doc_sort_index = np.argsort(np.array(query_doc_cos_dist))
            # retrievel top 3
            count = 0
            for rank, sort_index in enumerate(query_doc_sort_index):
                res.append([query_corpus[sort_index].split()[0],int(query_corpus[sort_index].split()[1])])
                if count == 3:
                    break
                else:
                    count += 1     
        return res
    
    else:
        return []

## Method2: BOW

In [ ]:
def retrieval_evidence_func2(query,page_keys,page_dictionary,load_processed_corpus_df):
    res = []
    # Determine if the first word in query is "There" and "A"
    if query.split()[0] == "There":
        query = query.replace("There","there") 
        
    if query.split()[0] == "A":
        query = query.replace("A","a") 
    
    # remove all 's 
    for word in query.split():
        if (word[-2:len(word)]) == "'s":
            query = query.replace(word,word[:-2])
            
     # remove all '
    for word in query.split():
        if (word[-1:len(word)]) == "'":
            query = query.replace(word,word[:-1])
            
    query_corpus = []
    if len(find_upper_word(query,page_keys)) >= 1:
        for query_word in find_upper_word(query,page_keys):
            if query_word[-1:len(query_word)] == ".":
                query_word = query_word[:-1]
            if query_word[-1:len(query_word)] == ",":
                query_word = query_word[:-1]
            query_corpus.extend(retrieve_sentenceText(query_word, page_keys,page_dictionary,load_processed_corpus_df))
        
        if len(query_corpus) == 1:
            res.append([query_corpus[0].split()[0],int(query_corpus[0].split()[1])])
        else:
            # preprocess claim
            processed_query_claim = pre_process(query)
            # preprocess evidences
            processed_query_corpus = []
            for corpus in query_corpus:
                processed_query_corpus.append(pre_process(corpus))

            count_vectorizer = CountVectorizer()

            processed_query_corpus.insert(0, processed_query_claim)

            count = count_vectorizer.fit_transform(processed_query_corpus)

            query_rep = count[0].todense()
            docs_rep = count[1:].todense()

            # calculate cosine similarity between the query and retrieved sentences
            query_doc_cos_dist = []

            # cosine distance, and hence no need to revese argsort result
            for doc_rep in docs_rep:
                query_doc_cos_dist.append(cosine(query_rep, doc_rep))

            query_doc_sort_index = np.argsort(np.array(query_doc_cos_dist))

            count = 0
            for rank, sort_index in enumerate(query_doc_sort_index):
                res.append([query_corpus[sort_index].split()[0],int(query_corpus[sort_index].split()[1])])
                if count == 2:
                    break
                else:
                    count += 1   
        return res
        
    else:
        return []

## Method3 TF IDF with threshold

In [ ]:
def retrieval_evidence_func3(query,page_keys,page_dictionary,load_processed_corpus_df):
    res = []
    # Determine if the first word in query is "There" and "A"
    if query.split()[0] == "There":
        query = query.replace("There","there") 
        
    if query.split()[0] == "A":
        query = query.replace("A","a") 
    
    # remove all 's 
    for word in query.split():
        if (word[-2:len(word)]) == "'s":
            query = query.replace(word,word[:-2])
            
     # remove all '
    for word in query.split():
        if (word[-1:len(word)]) == "'":
            query = query.replace(word,word[:-1])
            
    count = 0
    query_corpus = []

    if len(find_upper_word(query,page_keys)) >= 1:
        for query_word in find_upper_word(query,page_keys):
            if query_word[-1:len(query_word)] == ".":
                query_word = query_word[:-1]
            if query_word[-1:len(query_word)] == ",":
                query_word = query_word[:-1]
                
            query_corpus.extend(retrieve_sentenceText(query_word, page_keys,page_dictionary,load_processed_corpus_df))
        
        if len(query_corpus) == 0:
            return query_corpus
        
        if len(query_corpus) == 1:
            res.append([query_corpus[0].split()[0],int(query_corpus[0].split()[1])])
        else:
            # preprocess claim
            processed_query_claim = pre_process(query)
            # preprocess evidences
            processed_query_corpus = []
            for corpus in query_corpus:
                processed_query_corpus.append(pre_process(corpus))
            
            tfidf_vectorizer = TfidfVectorizer()
            processed_query_corpus.insert(0, processed_query_claim)

            tfidf = tfidf_vectorizer.fit_transform(processed_query_corpus)

            query_rep = tfidf[0].todense()
            docs_rep = tfidf[1:].todense()

            # calculate cosine similarity between the query and retrieved sentences
            query_doc_cos_dist = []

            # cosine distance, and hence no need to revese argsort result
            for doc_rep in docs_rep:
                query_doc_cos_dist.append(cosine(query_rep, doc_rep))

            query_doc_sort_index = np.argsort(np.array(query_doc_cos_dist))
            
            count = 0
            threshold = 0.0
#             best_query_corpus = []
            for rank, sort_index in enumerate(query_doc_sort_index):
                threshold = query_doc_cos_dist[sort_index] 
#                 best_query_corpus = [query_corpus[sort_index].split()[0],int(query_corpus[sort_index].split()[1])]
                if threshold > 0.78:
                    break
                else:
                    res.append([query_corpus[sort_index].split()[0],int(query_corpus[sort_index].split()[1])])                    
                    if count == 2:
                        break
                    else:
                        count += 1 
#                 res.append([query_corpus[sort_index].split()[0],int(query_corpus[sort_index].split()[1]),query_doc_cos_dist[sort_index]])
            
#             if len(res) == 0:
#                 res.append(best_query_corpus)
                
        return res
        
    else:
        return []

## Methond4 BERT

In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import Adam
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
nltk.download('punkt')

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from bert_serving.client import BertClient
bc = BertClient(check_length=False)

# for cased restart server with 
# bert-serving-start -model_dir /share/ShareFolder/cased_L-24_H-1024_A-16/ -cased_tokenization -max_batch_size=256 -gpu_memory_fraction=0.5 -num_worker=1
# for uncased restart server with 
# bert-serving-start -model_dir /share/ShareFolder/cased_L-24_H-1024_A-16/ -max_batch_size=256 -gpu_memory_fraction=0.9 -num_worker=1

def get_keras_model():
    model = Sequential()
    model.add(Dense(units=200, activation='relu', input_dim=3072))
    model.add(Dropout(0.3))
    model.add(Dense(units=50, activation='relu', input_dim=3072))
    model.add(Dropout(0.3))
    model.add(Dense(units=1, activation='sigmoid'))
    # optimizer = Adam(lr=0.01)
    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer='adam', metrics=['accuracy'])

    model.load_weights("best_weights_sentence.hdf5")
    return model

def get_features(claim_list, evidence_list, pair_list):
    claim_features = bc.encode(claim_list)
    evidence_features = bc.encode(evidence_list)
    pair_features = bc.encode(pair_list)
    return np.concatenate([claim_features, evidence_features, pair_features], axis=1)

keras_model = get_keras_model()


def retrieval_evidence_func4(query,page_keys,page_dictionary,load_processed_corpus_df):
    res = []
    # Determine if the first word in query is "There" and "A"
    if query.split()[0] == "There":
        query = query.replace("There","there") 
        
    if query.split()[0] == "A":
        query = query.replace("A","a") 
    
    # remove all 's 
    for word in query.split():
        if (word[-2:len(word)]) == "'s":
            query = query.replace(word,word[:-2])
            
     # remove all '
    for word in query.split():
        if (word[-1:len(word)]) == "'":
            query = query.replace(word,word[:-1])
            
    # remove all .
    for word in query.split():
        if (word[-1:len(word)]) == ".":
            query = query.replace(word,word[:-1])
    
    for word in query.split():
        if (word[-1:len(word)]) == ",":
            query = query.replace(word,word[:-1])
    
    count = 0
    query_corpus = []
    for query_word in find_upper_word(query,page_keys):
        query_corpus.extend(retrieve_sentenceText(query_word, page_keys,page_dictionary,load_processed_corpus_df))

    processed_query_claim = pre_process(query)
    # preprocess evidences
    claim_list = []
    evidence_list = []
    pair_list = []
    
    if len(query_corpus) == 0:
        return res
    
    for evidence in query_corpus:
        claim_list.append(processed_query_claim)
        evidence_list.append(evidence)
        pair_list.append(processed_query_claim + " ||| " + evidence)

    feature_matrix = get_features(claim_list, evidence_list, pair_list)
    labels = keras_model.predict(feature_matrix, batch_size=32, verbose=0)
    probabilities = []
    for label in labels:
        probabilities.append(label[0])

    # 排序, desending order
    query_doc_sort_index = np.argsort(probabilities)
    query_doc_sort_index[::-1]   # reverse

    count = 0
    for sort_index in query_doc_sort_index:
        if probabilities[sort_index] > 0.5:
            res.append([query_corpus[sort_index].split()[0],int(query_corpus[sort_index].split()[1])])
            count += 1
            if count >= 5:
                break

    return res

## Method5  TFIDF with new rules on brackets

In [ ]:
def retrieval_evidence_func5(query,page_keys,page_dictionary,load_processed_corpus_df):
    res = []
    # Determine if the first word in query is "There" and "A"
    if query.split()[0] == "There":
        query = query.replace("There","there") 
        
    if query.split()[0] == "A":
        query = query.replace("A","a") 
    
    # remove all 's 
    for word in query.split():
        if (word[-2:len(word)]) == "'s":
            query = query.replace(word,word[:-2])
            
     # remove all '
    for word in query.split():
        if (word[-1:len(word)]) == "'":
            query = query.replace(word,word[:-1])
            
    query_corpus = []
    if len(find_upper_word(query,page_keys)) >= 1:
        for query_word in find_upper_word(query,page_keys):
            if query_word[-1:len(query_word)] == ".":
                query_word = query_word[:-1]
            if query_word[-1:len(query_word)] == ",":
                query_word = query_word[:-1]
            
            if query_word in page_keys:
                retrieved_index = lrb_dictionary[query_word]

                for index in retrieved_index:
                    original_head = load_processed_corpus_df.loc[index, 'page_identifier'] 
                    if '-LRB' in original_head:
                        head_list = original_head[original_head.find('-LRB-')+len('-LRB-'):original_head.rfind('-RRB-')].split('_')
                        
                        for head in head_list:
                            if head in query:
                                query_corpus.append((load_processed_corpus_df.loc[index, 'text']))
                                break
                            else:
                                continue
                    else:
                        query_corpus.append((load_processed_corpus_df.loc[index, 'text']))
        
            
        if len(query_corpus) == 1:
            res.append([query_corpus[0].split()[0],int(query_corpus[0].split()[1])])
        else:
            # preprocess claim
            processed_query_claim = pre_process(query)
            # preprocess evidences
            processed_query_corpus = []
            for corpus in query_corpus:
                processed_query_corpus.append(pre_process(corpus))
            
            tfidf_vectorizer = TfidfVectorizer()
            processed_query_corpus.insert(0, processed_query_claim)

            tfidf = tfidf_vectorizer.fit_transform(processed_query_corpus)

            query_rep = tfidf[0].todense()
            docs_rep = tfidf[1:].todense()

            # calculate cosine similarity between the query and retrieved sentences
            query_doc_cos_dist = []
           # cosine distance, and hence no need to revese argsort result
            for doc_rep in docs_rep:
                query_doc_cos_dist.append(cosine(query_rep, doc_rep))
            
            query_doc_sort_index = np.argsort(np.array(query_doc_cos_dist))
            
            count = 0
            threshold = 0.0
            for rank, sort_index in enumerate(query_doc_sort_index):
                threshold = query_doc_cos_dist[sort_index] 
                if threshold > 0.78:
                    break
                else:
                    res.append([query_corpus[sort_index].split()[0],int(query_corpus[sort_index].split()[1])])                    
                    if count == 2:
                        break
                    else:
                        count += 1 

        return res
        
    else:
        return []


In [ ]:
from math import*
 
def euclidean_distance(x,y):
     return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))

## Method 6: Pure TF IDF 

In [10]:
def retrieval_evidence_func6(query,page_keys,page_dictionary,load_processed_corpus_df):
    res = []
    # Determine if the first word in query is "There" and "A"
    if query.split()[0] == "There":
        query = query.replace("There","there") 
        
    if query.split()[0] == "A":
        query = query.replace("A","a") 
    
    # remove all 's 
    for word in query.split():
        if (word[-2:len(word)]) == "'s":
            query = query.replace(word,word[:-2])
            
     # remove all '
    for word in query.split():
        if (word[-1:len(word)]) == "'":
            query = query.replace(word,word[:-1])
        
    query_corpus = []
    if len(find_upper_word(query,page_keys)) >= 1:
        for query_word in find_upper_word(query,page_keys):
            if query_word[-1:len(query_word)] == ".":
                query_word = query_word[:-1]
            if query_word[-1:len(query_word)] == ",":
                query_word = query_word[:-1]
            
            query_corpus.extend(retrieve_sentenceText(query_word, page_keys,page_dictionary,load_processed_corpus_df))
        
        if len(query_corpus) == 1:
            res.append([query_corpus[0].split()[0],int(query_corpus[0].split()[1])])
        else:
            # preprocess claim
            processed_query_claim = pre_process(query)
            # preprocess evidences
            processed_query_corpus = []
            for corpus in query_corpus:
                processed_query_corpus.append(pre_process(corpus))
            
            tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
            processed_query_corpus.insert(0, processed_query_claim)

            tfidf = tfidf_vectorizer.fit_transform(processed_query_corpus)

            query_rep = tfidf[0].todense()
            docs_rep = tfidf[1:].todense()
                        # calculate cosine similarity between the query and retrieved sentences
            query_doc_cos_dist = []

            # cosine distance, and hence no need to revese argsort result
            for doc_rep in docs_rep:
                query_doc_cos_dist.append(cosine(query_rep, doc_rep))
#                 query_doc_cos_dist.append(euclidean_distance(query_rep, doc_rep))
                
            query_doc_sort_index = np.argsort(np.array(query_doc_cos_dist))

            count = 0
            for rank, sort_index in enumerate(query_doc_sort_index):
                res.append([query_corpus[sort_index].split()[0],int(query_corpus[sort_index].split()[1])])
                if count == 4:
                    break
                else:
                    count += 1   
        return res
        
    else:
        return []

## Testing

In [ ]:
# testing
test_claim1 = "Brad Wilk helped co-found Rage in 1962."
test_claim2 = "The Faroe Islands are no longer part of the Kingdom of Mercia."
test_claim3 = "Down With Love is a 2003 comedy film."
test_claim4 = "Telemundo is a English-language television network."
# To test LRB
test_claim5 = 'Hourglass is performed by a Russian singer-songwriter.'
# To test first word "There"
test_claim6 = "There are no musical or creative works in existence that have been created by Phillip Glass."
# To test 's ' in the word
test_claim7 = "Damon Albarn's debut album was released in 2011."

test_claim8 = "Temple of the Dog celebrated the 37th anniversary of their self-titled album."
test_claim9 = "Savages was exclusively a German film."
test_claim10 = "L.A. Reid has served as the president of a record label."
test_claim11 = "Bermuda Triangle is in the western part of the Himalayas."

print(retrieval_evidence_func6(test_claim1,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim2,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim3,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim4,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim5,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim6,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim7,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim8,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim9,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim10,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func6(test_claim11,page_keys,lrb_dictionary,load_processed_corpus_df))

print("\n")

print(retrieval_evidence_func7(test_claim1,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim2,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim3,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim4,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim5,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim6,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim7,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim8,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim9,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim10,page_keys,lrb_dictionary,load_processed_corpus_df))
print(retrieval_evidence_func7(test_claim11,page_keys,lrb_dictionary,load_processed_corpus_df))

## Write result to  devset file.

In [ ]:
global_test_data=Manager().dict(res_data)   

i = multiprocessing.Value("i",0)
def deal_dataset(key):
    detail_dict = global_test_data[key]
    detail_dict["evidence"] = []
    query = detail_dict["claim"]
#     if detail_dict["label"] == "NOT ENOUGH INFO":
#         detail_dct["evidence"] = []
#     else:
#     detail_dict["evidence"] = retrieval_evidence_func3(query,page_keys,lrb_dictionary,load_processed_corpus_df)

    detail_dict["evidence"] = retrieval_evidence_func6(query,page_keys,lrb_dictionary,load_processed_corpus_df)
    global_test_data[key] = detail_dict
    i.value += 1
    print(i.value)

pool=Pool(processes = 10)
for key in list(res_data):
    pool.apply_async(deal_dataset, args=(key,))

pool.close()
pool.join()  

with open('result_dev_526_1_tfidf.json', 'w') as f:
    json.dump(dict(global_test_data), f, indent = 4)

## Write result to test file.

In [ ]:
global_test_data=Manager().dict(test_data)    

i = multiprocessing.Value("i",0)
def deal_dataset(key):
    detail_dict = global_test_data[key]
    detail_dict["evidence"] = []
    query = detail_dict["claim"]
    detail_dict["evidence"] = retrieval_evidence_func6(query,page_keys,lrb_dictionary,load_processed_corpus_df)
    global_test_data[key] = detail_dict
    i.value += 1
    print(i.value)

pool=Pool(processes = 10)
for key in list(test_data):
    pool.apply_async(deal_dataset, args=(key,))

pool.close()
pool.join()  

with open('result_test_524_tfidf5.json', 'w') as f:
    json.dump(dict(global_test_data), f, indent = 4)

## Write training data result to file.

In [ ]:
with open('train.json', 'r') as f9:  # load dev dataset
        train_data = json.load(f9) 
print("Length of the train data is: " + str(len(train_data)))

with open('train_result.json', 'r') as f8:  # store result 
        train_res_data = json.load(f8) 
print("Length of the train result data is: " + str(len(train_res_data)))

In [ ]:
global_test_data=Manager().dict(train_res_data)   

i = multiprocessing.Value("i",0)
def deal_dataset(key):
    detail_dict = global_test_data[key]
    detail_dict["evidence"] = []
    query = detail_dict["claim"]
#     if detail_dict["label"] == "NOT ENOUGH INFO":
#         detail_dct["evidence"] = []
#     else:
#     detail_dict["evidence"] = retrieval_evidence_func3(query,page_keys,lrb_dictionary,load_processed_corpus_df)

    detail_dict["evidence"] = retrieval_evidence_func5(query,page_keys,lrb_dictionary,load_processed_corpus_df)
    global_test_data[key] = detail_dict
    i.value += 1
    print(i.value)

pool=Pool(processes = 10)
# first 50000 in train data
for key in list(train_res_data)[:50000]:
    pool.apply_async(deal_dataset, args=(key,))

pool.close()
pool.join()  

with open('result_train_522_tfidf3.json', 'w') as f:
    json.dump(dict(global_test_data), f, indent = 4)